In [1]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

parent_llm = ChatOpenAI(model='gpt-4o-mini')
subgraph_llm = ChatOpenAI(model='gpt-4o')

In [2]:
class ParentState(TypedDict):

    question: str
    answer_eng: str
    answer_hin: str

In [3]:
def translate_text(state: ParentState):

    prompt = f"""
Translate the following text to Bengali.
Keep it natural and clear. Do not add extra content.

Text:
{state["answer_eng"]}
""".strip()
    
    translated_text = subgraph_llm.invoke(prompt).content

    return {'answer_hin': translated_text}

In [4]:
subgraph_builder = StateGraph(ParentState)

subgraph_builder.add_node('translate_text', translate_text)

subgraph_builder.add_edge(START, 'translate_text')
subgraph_builder.add_edge('translate_text', END)

subgraph = subgraph_builder.compile()

In [ ]:
def generate_answer(state: ParentState):

    answer = parent_llm.invoke(f"You are a helpful  assistant. Answer clearly.\n\nQuestion: {state['question']}").content
    return {'answer_eng': answer}

In [9]:
parent_builder = StateGraph(ParentState)

parent_builder.add_node("answer", generate_answer)
parent_builder.add_node("translate", subgraph)

parent_builder.add_edge(START, 'answer')
parent_builder.add_edge('answer', 'translate')
parent_builder.add_edge('translate', END)

graph = parent_builder.compile()

In [ ]:
graph.invoke({'question': 'What   is quantum physics'})

{'question': 'What is quantum physics',
 'answer_eng': "Quantum physics, also known as quantum mechanics, is a fundamental branch of physics that deals with the behavior of matter and energy at the smallest scales, typically at the level of atoms and subatomic particles. It describes how particles such as electrons and photons behave, often in ways that are very different from classical physics.\n\nKey concepts in quantum physics include:\n\n1. **Wave-Particle Duality**: Particles exhibit both wave-like and particle-like properties. For example, electrons can show interference patterns like waves but also exist as discrete particles.\n\n2. **Quantization**: Energy levels in quantum systems are quantized, meaning they can only take on specific values rather than a continuous range.\n\n3. **Uncertainty Principle**: Formulated by Werner Heisenberg, it states that certain pairs of physical properties, like position and momentum, cannot both be precisely determined at the same time.\n\n4. *